# Countertop detection

This notebook will be full loop from training dataset examination and annotation through segmentation training.

1. Countertop Mask training data examination. 
2. Model training (really just a sanity check since we expect to really run this at the commandline. 
3. Segmentation Detection Evaluation. 


In [ ]:
%matplotlib inline
import os
import sys
import math,copy
import skimage.io
import time, random
import numpy as np
import json, io
import pandas as pd
import glob
import PIL
import cv2 as cv
from IPython.display import HTML
from base64 import b64encode, b64decode




import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow



ms_udfix = lambda x:x.replace('\\', '/')[:-1]

In [ ]:
import keras
from importlib import reload
from keras.engine import topology
keras.__version__

ROOT_DIR = ms_udfix(r"C:\Users\dominics.CORP\git\Mask_RCNN\\")
print(ROOT_DIR)
sys.path.append(ROOT_DIR)
sys.path.append(ROOT_DIR+'samples/coco/')#for loading coco.py

import coco
import mrcnn.utils
import mrcnn.model as modellib
import mrcnn.visualize

from ds_utils.pil import *
from ds_utils.geometry import *

## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [ ]:
#I/O

V_ROOT_DIR = ms_udfix(r"C:\Users\Public\Mask_RCNN-master\\")

# Root directory of the project
#See above__ROOT_DIR = ms_udfix(r"C:\Users\Public\Mask_RCNN-master\\")

# Directory to save logs and trained model
# MODEL_DIR = os.path.join(ms_udfix(r"C:\Users\Public\Segmentation\seg-countertops\\"), "logs/")
# if not(os.path.exists(MODEL_DIR)):os.mkdir(MODEL_DIR)
#MODEL_DIR = os.path.join(ms_udfix(r"C:\Users\Public\Mask_RCNN-master\\"), "logs")

out_ud = os.path.join(ms_udfix(r"C:\Users\dominics.CORP\Documents\dev\MaskRCNN_ds\model_logs\\"))
if not(os.path.exists(out_ud)):
    os.mkdir(out_ud)

TrainingData_fn = "1106d_HL_data.json"
traindata_path = os.path.join(ms_udfix(r"C:\Users\Public\Mask_RCNN-master\segmentation\kitchen\annotations\\"), TrainingData_fn)
# Path to trained weights file
# Download this file and place in the root of your 
# project (See README file for details)
if None:
    model_fn="mask_rcnn_coco_0026.h5"
    ds_COCO_MODEL_PATH = os.path.join(ms_udfix(r"C:\Users\dominics.CORP\Documents\dev\MaskRCNN_ds\model_logs\coco20181106T1311\\"), model_fn)

MODEL_DIR = ms_udfix(r"C:\Users\dominics.CORP\Documents\dev\MaskRCNN_ds\model_logs\coco20181108T0112\\")
m_fn = "mask_rcnn_coco_0050.h5"
m_1106 = os.path.join(MODEL_DIR,m_fn)

vignesh_COCO_MODEL_PATH = os.path.join(V_ROOT_DIR, "mask_rcnn_coco1_0006.h5")
COCO_MODEL_PATH = m_1106#ds_COCO_MODEL_PATH




#Train against eval:
if True:
    IMGS_TO_TEST=None
    train_ud = ms_udfix(r"C:\Users\Public\Segmentation\seg-countertops\annotationsWithImages\\")
    IMAGE_DIR = train_ud
    eval_type = "train_eval"
#Eval against Test:
if None:
    IMGS_TO_TEST=100
    train_ud = ms_udfix(r"C:\Users\Public\Mask_RCNN-master\img_data\images\\")
    IMAGE_DIR = train_ud##os.path.join(ROOT_DIR, "graniteimages")
    eval_type = "test_eval"

_dir,model_name = os.path.split(os.path.dirname(COCO_MODEL_PATH))


OUT_annot_path = out_ud+'%s-%s.json'%(model_name, eval_type)
print('COCO_MODEL_PATH:', COCO_MODEL_PATH, "model_name: ", model_name)
print('OUT_annot_path', OUT_annot_path)

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    #NUM_CLASSES=2

config = InferenceConfig()
config.display()
#class_names=['','countertop']

## Create Model and Load Trained Weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
#model.load_weights(COCO_MODEL_PATH, by_name=True)
model.load_weights(COCO_MODEL_PATH, by_name=True)#,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])
config.display()

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [ ]:
import ds_utils.json_templates as t
print(dir(t))
region_template=0
from ds_utils.json_templates import *

print(region_template)

In [ ]:
from ds_utils.json_templates import *


def masks_to_poly(r, class_names):
    """ rois (5, 4)
        scores (5,)
        masks (768, 1024, 5)
        class_ids (5,)"""
    masks = np.array(r['masks'])
    class_ids = np.array(r['class_ids'])
    scores = np.array(r['scores'])
    row,col, num_msks = masks.shape
    class_ids = r['class_ids']
    d = {'mask':[], 'img':[], 'call':[], 'score':[], 'pLst':[]}
    num_calls = 0
    for c in range(num_msks):
        msk = masks[:,:,c]
        if np.sum(msk)>0:
            d['pLst'].append(grid_simplify(MaskToPointLst(msk), resolution=30))
            d['mask'].append(msk)
            d['call'].append(class_names[class_ids[c]])
            d['score'].append(scores[c])
            num_calls +=1
    if num_calls:return d
    return None

In [ ]:
category_map = {'bg':0}

for c in train_data["categories"]:
    c_id = c["id"]
    c_name = c["name"]
    if not(c_name in category_map):category_map[c_name]=c_id
    if (category_map[c_name]!=c_id):print("Error building categoryMap")
c = list(category_map.items())
c.sort(key=lambda x:x[1])
class_names =  [c[0] for c in c]
print(class_names, category_map)

#Build up Training data from before:
print(traindata_path)
source_name = os.path.basename(traindata_path)
train_data = json.loads(open(traindata_path).read())
image_idTofilename = {}
for img_rec in train_data['images']:
    image_idTofilename[img_rec['id']] = img_rec['file_name']
fn_to_annotLst ={}
for anot in train_data['annotations']:
    fn = image_idTofilename[anot['image_id']]
    if not(fn in fn_to_annotLst):fn_to_annotLst[fn] = []
    region = copy.deepcopy(region_template)
    region["region_attributes"]["call_type"]="train"
    region["region_attributes"]["call"]=class_names[anot["category_id"]]
    region["region_attributes"]["model_call"]=""
    region["region_attributes"]["source"]=source_name
    p = anot["segmentation"][0]
    for i in range(0, len(p)-1,2):
        region["shape_attributes"]["all_points_x"].append(int(p[i]))
        region["shape_attributes"]["all_points_y"].append(int(p[i+1]))
    fn_to_annotLst[fn].append(region)
 



In [ ]:
#CALL then save to JSON format

via_project = copy.deepcopy(via_project_TEMPLATE)
via_project["_via_settings"]["project"]["name"]=model_name
via_project['_via_attributes']=Make_via_attributes_template()
count = 0
in_fnLst = glob.glob(train_ud+"*.jpg")
if IMGS_TO_TEST:
    random.shuffle(in_fnLst)
    in_fnLst = in_fnLst[:IMGS_TO_TEST]
in_fnLst.sort()
for image_path in in_fnLst:
    img_fn = os.path.basename(image_path)
    image = skimage.io.imread(image_path)
    file_sz = image.size
    #if count==2:break
    count+=1
    # Run detection
    print('image_path', image_path)
    results = model.detect([image], verbose=1) 
    print( type(image), image.size, 'num_results: %i'%len(results))
    for r in results:
        d = masks_to_poly(r, class_names)
        if d==None:
            print('no calls for ', img_fn)
            continue
        print (img_fn)
        x = Make_via_filename_template(img_fn, d['pLst'], scoreLst=d['score'], callLst=d['call'], file_sz = file_sz)
        via_project["_via_img_metadata"][img_fn]=x
        #Add previous training annotations
        if img_fn in fn_to_annotLst:
            via_project["_via_img_metadata"][img_fn]["regions"].extend(fn_to_annotLst[img_fn])
            

with open(OUT_annot_path, 'w') as outfile:
    json.dump(via_project, outfile)

In [ ]:
OUT_annot_path

## Build Training Set using annotated file  
training set form:
    `{
        "images": [],
    "info": {
        "url": "url",
        "contributor": "mls",
        "description": "kitchen",
        "date_created": "",
        "year": 0,
        "version": "1.0"
    },
    "type": "instances",
    "categories": [],
    "annotations": [],
    "licenses": [{
                "url": "http:\\/\\/creativecommons.org\\/licenses\\/by-nc-sa\\/2.0\\/",
                "id": 1,
                "name": "Attribution-NonCommercial-ShareAlike License"
            }]
        }`

In [ ]:
import ds_utils.json_templates as J
reload(J)
annot_path = ms_udfix(r"C:\Users\dominics.CORP\Downloads\\") + "1106d_handlabelled (2).json"
train_out_path = os.path.join(ms_udfix(r"C:\Users\Public\Mask_RCNN-master\segmentation\kitchen\annotations\\"), "1106d_HL_data.json")


print(annot_path)
via_project = json.loads(open(annot_path,"r").read())
project_name = via_project["_via_settings"]["project"]["name"]=model_name
img_dict = via_project["_via_img_metadata"]

TrainSet = J.Make_TrainingSet_Template()

Acceptable_CallLst = ['counter_top', 'floor', 'sink', 'stove']
call_counter = {}
image_id = 0
for fn, file_rec in img_dict.items():
    annotations = []
    regions = file_rec['regions']
    print(fn, 'num_regions', len(regions))
    annot_id=0
    for r in regions:
        call = r["region_attributes"]["call"]
        num_points=len(r["shape_attributes"]["all_points_x"])
        if (num_points<4) or not(call in Acceptable_CallLst):
            print('skipping call:', call, 'num_points', num_points)
            continue
        if not(call in call_counter):call_counter[call]=0
        call_counter[call]+=1
        #Add annotation
        category_id = Acceptable_CallLst.index(call)+1
        sLst= (r["shape_attributes"]["all_points_x"], r["shape_attributes"]["all_points_y"])
        A = J.Make_tsAnnotation(annot_id, image_id, sLst, category_id)
        
        annotations.append(A)
        annot_id+=1
    if annot_id>0:
        TrainSet['images'].append(J.Make_tsImagerec(fn, image_id))
        TrainSet['annotations'].extend(annotations)
        TrainSet['categories']#.append(J.Make_tsCategory("unknown", 0, supercategory="kitchen"))
    for i in range(len(Acceptable_CallLst)):
        TrainSet['categories'].append(J.Make_tsCategory(Acceptable_CallLst[i], i+1, supercategory="kitchen"))
    image_id+=1
    
print(call_counter)


with open(train_out_path, 'w') as outfile:
    json.dump(TrainSet, outfile)

In [ ]:
print(r)

In [ ]:
#Load images from json files:
train_ud = ms_udfix(r"C:\Users\Public\Segmentation\seg-countertops\annotationsWithImages\\")
imgjson_files = [os.path.join(train_ud, pos_json) for pos_json in os.listdir(train_ud) if pos_json.endswith('.json')]
print(train_ud)
jipath = random.choice(imgjson_files)
if 1:
    ipath = jipath.replace('.json', '.jpg')
    pil_im = Image.open(ipath)
    display(pil_im)
PlotPoly(jipath)